In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Jun 21 09:09:32 2023

@author: alex
"""
import numpy as np
import pandas as pd 
import scipy.stats as st
from scipy.stats import norm

models = './models/'
models_res = pd.read_csv(f'{models}Multi channel models diff analysis all 230725.csv')
# models_res = pd.read_csv(f'{models}Multi channel models diff analysis.csv')
columns = models_res.columns .tolist()

print(columns)
print(models_res.shape)

['Model', 'Image size', 'Batch Size', 'N Channels', 'CV score', 'Diff CV', 'Diff CV %%', 'PrLB score', 'Diff Private LB', 'Diff Private LB %%', 'PubLB score', 'Diff Public LB', 'Diff Public LB %%', 'CV 5 channels', 'Private LB 5 ch', 'Public LB 5 ch', 'CV 4 channels', 'Private LB 4 ch', 'Public LB 4 ch', 'Remarks']
(133, 20)


In [2]:
models_res.head(10)

,Model,Image size,Batch Size,N Channels,CV score,Diff CV,Diff CV %%,PrLB score,Diff Private LB,Diff Private LB %%,PubLB score,Diff Public LB,Diff Public LB %%,CV 5 channels,Private LB 5 ch,Public LB 5 ch,CV 4 channels,Private LB 4 ch,Public LB 4 ch,Remarks
0,EfficientNet B0 v2,128.0,128.0,3,0.8310,NaN,NaN,0.8111,NaN,NaN,0.8278,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RGB
1,EfficientNet B0 v2,128.0,128.0,MI,0.8290,-0.0020,-0.24%,0.8115,0.0005,0.06%,0.8339,0.0061,0.73%,NaN,NaN,NaN,-0.00241,0.00062,0.00732,RGB - MI
2,EfficientNet B0 v2,128.0,128.0,5,0.8410,0.0100,1.19%,0.8260,0.0150,1.82%,0.8398,0.0120,1.43%,0.01189,0.01816,0.01429,NaN,NaN,NaN,RGB - MI - EI
3,EfficientNet B0 v2,128.0,128.0,3,0.8448,NaN,NaN,0.8209,NaN,NaN,0.8116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RGB
4,EfficientNet B0 v2,128.0,128.0,MI,0.8338,-0.0110,-1.32%,0.8161,-0.0048,-0.59%,0.8405,0.0289,3.44%,NaN,NaN,NaN,-0.01319,-0.00588,0.03438,RGB - MI
5,EfficientNet B0 v2,128.0,128.0,5,0.8347,-0.0101,-1.21%,0.8203,-0.0006,-0.07%,0.8350,0.0234,2.80%,-0.01210,-0.00073,0.02802,NaN,NaN,NaN,RGB - MI - EI
6,EfficientNet B0 v2,256.0,64.0,3,0.8543,NaN,NaN,0.8352,NaN,NaN,0.8538,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RGB
7,EfficientNet B0 v2,256.0,64.0,MI,0.8544,0.0001,0.01%,0.8423,0.0071,0.84%,0.8677,0.0139,1.60%,NaN,NaN,NaN,0.00012,0.00843,0.01602,RGB - MI
8,EfficientNet B0 v2,256.0,64.0,5,0.8613,0.0070,0.81%,0.8643,0.0291,3.37%,0.8596,0.0058,0.67%,0.00813,0.03367,0.00675,NaN,NaN,NaN,RGB - MI - EI
9,EfficientNet B0 v2,512.0,64.0,3,0.8609,NaN,NaN,0.8569,NaN,NaN,0.8640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,RGB


In [3]:
models_res_EI = models_res.loc[models_res['N Channels'] == 'EI']
print(f'Number of the models with EI index: {models_res_EI.shape[0]}')
models_res_MI = models_res.loc[models_res['N Channels'] == 'MI']
print(f'Number of the models with MI index: {models_res_MI.shape[0]}')

models_res_5_all = models_res.loc[models_res['N Channels'] == '5']
print(f'Total number of the models with 5 channels: {models_res_5_all.shape[0]}')
models_res_5 = models_res_5_all.copy()

Number of the models with EI index: 35
Number of the models with MI index: 18
Number of the models with 5 channels: 40


In [5]:
def MI_EI_order(x):
    tt = x['Remarks'].split(' ')
    return tt.index('MI') > tt.index('EI')

In [7]:
models_res_5['EI_first'] = models_res_5.apply(lambda x: MI_EI_order(x), axis = 1)

In [26]:
MI_CV5_channels = models_res_5.loc[models_res_5['EI_first'] == False, ['CV 5 channels']].values.squeeze()
MI_Pub5_channels = models_res_5.loc[models_res_5['EI_first'] == False, ['Public LB 5 ch']].values.squeeze()
MI_Prv5_channels = models_res_5.loc[models_res_5['EI_first'] == False, ['Private LB 5 ch']].values.squeeze()

EI_CV5_channels = models_res_5.loc[models_res_5['EI_first'] == True, ['CV 5 channels']].values.squeeze()
EI_Pub5_channels = models_res_5.loc[models_res_5['EI_first'] == True, ['Public LB 5 ch']].values.squeeze()
EI_Prv5_channels = models_res_5.loc[models_res_5['EI_first'] == True, ['Private LB 5 ch']].values.squeeze()

In [38]:
MI_CV4_channels = models_res_MI['CV 4 channels'].dropna().values
MI_Pub4_channels = models_res_MI['Public LB 4 ch'].dropna().values
MI_Prv4_channels = models_res_MI['Private LB 4 ch'].dropna().values

EI_CV4_channels = models_res_EI['CV 4 channels'].dropna().values
EI_Pub4_channels = models_res_EI['Public LB 4 ch'].dropna().values
EI_Prv4_channels = models_res_EI['Private LB 4 ch'].dropna().values

In [63]:
#create 95% confidence interval for current set of models mean AUC gain

ci_CV5_MI = st.t.interval(0.95, df=len(MI_CV5_channels)-1, loc=np.mean(MI_CV5_channels), scale=st.sem(MI_CV5_channels)) 
print(f'Mean of AUC gain in cross-validation 5ch with MI index first vs 3ch is {np.mean(MI_CV5_channels) * 100:.2f}%. 95% Confidence interval is ({ci_CV5_MI[0] * 100:.2f}%, {ci_CV5_MI[1] * 100:.2f}%)')
ci_CV5_EI = norm.interval(0.95, loc=np.mean(EI_CV5_channels), scale=st.sem(EI_CV5_channels)) 
print(f'Mean of AUC gain in cross-validation 5ch with EI index first vs 3ch is {np.mean(EI_CV5_channels) * 100:.2f}%. 95% Confidence interval is ({ci_CV5_EI[0] * 100:.2f}%, {ci_CV5_EI[1] * 100:.2f}%)')
ci_Pub5_MI = st.t.interval(0.95, df=len(MI_Pub5_channels)-1, loc=np.mean(MI_Pub5_channels), scale=st.sem(MI_Pub5_channels)) 
print(f'Mean of AUC gain in validation on Kaggle Public test set 5ch with MI index first vs 3ch is {np.mean(MI_Pub5_channels) * 100:.2f}%. 95% Confidence interval is ({ci_Pub5_MI[0] * 100:.2f}%, {ci_Pub5_MI[1] * 100:.2f}%)')
ci_Pub5_EI = st.t.interval(0.95, df=len(EI_Pub5_channels)-1, loc=np.mean(EI_Pub5_channels), scale=st.sem(EI_Pub5_channels)) 
print(f'Mean of AUC gain in validation on Kaggle Public test set 5ch with EI index first vs 3ch is {np.mean(EI_Pub5_channels) * 100:.2f}%. 95% Confidence interval is ({ci_Pub5_EI[0] * 100:.2f}%, {ci_Pub5_EI[1] * 100:.2f}%)')
ci_Prv5_MI = st.t.interval(0.95, df=len(MI_Prv5_channels)-1, loc=np.mean(MI_Prv5_channels), scale=st.sem(MI_Prv5_channels)) 
print(f'Mean of AUC gain in validation on Kaggle Private test set n 5ch  with MI index first vs 3ch is {np.mean(MI_Prv5_channels) * 100:.2f}%. 95% Confidence interval is ({ci_Prv5_MI[0] * 100:.2f}%, {ci_Prv5_MI[1] * 100:.2f}%)')
ci_Prv5_EI = st.t.interval(0.95, df=len(EI_Prv5_channels)-1, loc=np.mean(EI_Prv5_channels), scale=st.sem(EI_Prv5_channels)) 
print(f'Mean of AUC gain in validation on Kaggle Private test set n 5ch  with EI index first vs 3ch is {np.mean(EI_Prv5_channels) * 100:.2f}%. 95% Confidence interval is ({ci_Prv5_EI[0] * 100:.2f}%, {ci_Prv5_EI[1] * 100:.2f}%)\n')

ci_CV4_MI = st.t.interval(0.95, df=len(MI_CV4_channels)-1, loc=np.mean(MI_CV4_channels), scale=st.sem(MI_CV4_channels)) 
print(f'Mean of AUC gain in cross-validation 4ch with MI vs 3ch is {np.mean(MI_CV4_channels) * 100:.2f}%. 95% Confidence interval is ({ci_CV4_MI[0] * 100:.2f}%, {ci_CV4_MI[1] * 100:.2f}%)')
ci_CV4_EI = st.t.interval(0.95, df=len(EI_CV4_channels)-1, loc=np.mean(EI_CV4_channels), scale=st.sem(EI_CV4_channels)) 
print(f'Mean of AUC gain in cross-validation 4ch with EI vs 3ch is {np.mean(EI_CV4_channels) * 100:.2f}%. 95% Confidence interval is ({ci_CV4_EI[0] * 100:.2f}%, {ci_CV4_EI[1] * 100:.2f}%)')
ci_Pub4_MI = st.t.interval(0.95, df=len(MI_Pub4_channels)-1, loc=np.mean(MI_Pub4_channels), scale=st.sem(MI_Pub4_channels)) 
print(f'Mean of AUC gain in validation on Kaggle Public test set 4ch with MI vs 3ch is {np.mean(MI_Pub4_channels) * 100:.2f}%. 95% Confidence interval is ({ci_Pub4_MI[0] * 100:.2f}%, {ci_Pub4_MI[1] * 100:.2f}%)')
ci_Pub4_EI = st.t.interval(0.95, df=len(EI_Pub4_channels)-1, loc=np.mean(EI_Pub4_channels), scale=st.sem(EI_Pub4_channels)) 
print(f'Mean of AUC gain in validation on Kaggle Public test set 4ch with EI vs 3ch is {np.mean(EI_Pub4_channels) * 100:.2f}%. 95% Confidence interval is ({ci_Pub4_EI[0] * 100:.2f}%, {ci_Pub4_EI[1] * 100:.2f}%)')
ci_Prv4_MI = st.t.interval(0.95, df=len(MI_Prv4_channels)-1, loc=np.mean(MI_Prv4_channels), scale=st.sem(MI_Prv4_channels)) 
print(f'Mean of AUC gain in validation on Kaggle Private test set  4ch vs 3ch is {np.mean(MI_Prv4_channels) * 100:.2f}%. 95% Confidence interval is ({ci_Prv4_MI[0] * 100:.2f}%, {ci_Prv4_MI[1] * 100:.2f}%)')
ci_Prv4_EI = st.t.interval(0.95, df=len(EI_Prv4_channels)-1, loc=np.mean(EI_Prv4_channels), scale=st.sem(EI_Prv4_channels)) 
print(f'Mean of AUC gain in validation on Kaggle Private test set  4ch vs 3ch is {np.mean(EI_Prv4_channels) * 100:.2f}%. 95% Confidence interval is ({ci_Prv4_EI[0] * 100:.2f}%, {ci_Prv4_EI[1] * 100:.2f}%)\n')

# all_5 = pd.concat([CV5_channels, Pub5_channels, Prv5_channels])
all_5 = np.concatenate([MI_CV5_channels, MI_Pub5_channels, MI_Prv5_channels, EI_CV5_channels, EI_Pub5_channels, EI_Prv5_channels])
print(f'All 3 test sets combined, number of data points: {all_5.shape[0]}')
ci_all5 = st.t.interval(0.95, df=len(all_5)-1, loc=np.mean(all_5), scale=st.sem(all_5)) 
print(f'Mean of AUC gain in validation 5ch vs 3ch is {np.mean(all_5) * 100:.2f}%. 95% Confidence interval is ({ci_all5[0] * 100:.2f}%, {ci_all5[1] * 100:.2f}%)')
ci_all5_norm = st.norm.interval(0.95, loc=np.mean(all_5), scale=st.sem(all_5))
print(f'Mean of AUC gain in validation 5ch vs 3ch is {np.mean(all_5) * 100:.2f}%. 95% Confidence interval norm is ({ci_all5_norm[0] * 100:.2f}%, {ci_all5_norm[1] * 100:.2f}%)\n')

# all_5_MI = np.concatenate([MI_CV5_channels, MI_Pub5_channels, MI_Prv5_channels])
# print(all_5_MI.shape[0])
# ci_all5_MI = st.t.interval(0.95, df=len(all_5_MI)-1, loc=np.mean(all_5_MI), scale=st.sem(all_5_MI)) 
# print(f'Mean of AUC gain in validation on all of 3 test sets 5ch vs 3ch is {np.mean(all_5_MI) * 100:.2f}%. 95% Confidence interval is ({ci_all5_MI[0] * 100:.2f}%, {ci_all5_MI[1] * 100:.2f}%)')
# ci_all5_norm_MI = st.norm.interval(0.95, loc=np.mean(all_5_MI), scale=st.sem(all_5_MI))
# print(f'Mean of AUC gain in validation on all of 3 test sets 5ch vs 3ch is {np.mean(all_5_MI) * 100:.2f}%. 95% Confidence interval norm is ({ci_all5_norm_MI[0] * 100:.2f}%, {ci_all5_norm_MI[1] * 100:.2f}%)')
# all_5_EI = np.concatenate([EI_CV5_channels, EI_Pub5_channels, EI_Prv5_channels])
# print(all_5_EI.shape[0])
# ci_all5_EI = st.t.interval(0.95, df=len(all_5_EI)-1, loc=np.mean(all_5_EI), scale=st.sem(all_5_EI)) 
# print(f'Mean of AUC gain in validation on all of 3 test sets 5ch vs 3ch is {np.mean(all_5_EI) * 100:.2f}%. 95% Confidence interval is ({ci_all5_EI[0] * 100:.2f}%, {ci_all5_EI[1] * 100:.2f}%)')
# ci_all5_norm_EI = st.norm.interval(0.95, loc=np.mean(all_5_EI), scale=st.sem(all_5_EI))
# print(f'Mean of AUC gain in validation on all of 3 test sets 5ch vs 3ch is {np.mean(all_5_EI) * 100:.2f}%. 95% Confidence interval norm is ({ci_all5_norm_EI[0] * 100:.2f}%, {ci_all5_norm_EI[1] * 100:.2f}%)\n')

all_4_MI = np.concatenate([MI_CV4_channels, MI_Pub4_channels, MI_Prv4_channels])
print(f'All 3 test sets combined when MI is used as 4th channel, number of data points: {all_4_MI.shape[0]}')
ci_all4_MI = st.t.interval(0.95, df=len(all_4_MI)-1, loc=np.mean(all_4_MI), scale=st.sem(all_4_MI)) 
print(f'Mean of AUC gain in validation 4ch with MI vs 3ch is {np.mean(all_4_MI) * 100:.2f}%. 95% Confidence interval is ({ci_all4_MI[0] * 100:.2f}%, {ci_all4_MI[1] * 100:.2f}%)')
ci_all4_norm_MI = st.norm.interval(0.95, loc=np.mean(all_4_MI), scale=st.sem(all_4_MI))
print(f'Mean of AUC gain in validation 4ch with MI vs 3ch is {np.mean(all_4_MI) * 100:.2f}%. 95% Confidence interval norm is ({ci_all4_norm_MI[0] * 100:.2f}%, {ci_all4_norm_MI[1] * 100:.2f}%)')
all_4_EI = np.concatenate([EI_CV4_channels, EI_Pub4_channels, EI_Prv4_channels])
print(f'All 3 test sets combined when EI is used as 4th channel, number of data points: {all_4_EI.shape[0]}')
ci_all4_EI = st.t.interval(0.95, df=len(all_4_EI)-1, loc=np.mean(all_4_EI), scale=st.sem(all_4_EI)) 
print(f'Mean of AUC gain in validation 4ch with EI vs 3ch is {np.mean(all_4_EI) * 100:.2f}%. 95% Confidence interval is ({ci_all4_EI[0] * 100:.2f}%, {ci_all4_EI[1] * 100:.2f}%)')
ci_all4_norm_EI = st.norm.interval(0.95, loc=np.mean(all_4_EI), scale=st.sem(all_4_EI))
print(f'Mean of AUC gain in validation 4ch with EI vs 3ch is {np.mean(all_4_EI) * 100:.2f}%. 95% Confidence interval norm is ({ci_all4_norm_EI[0] * 100:.2f}%, {ci_all4_norm_EI[1] * 100:.2f}%)\n')

Mean of AUC gain in cross-validation 5ch with MI index first vs 3ch is 0.44%. 95% Confidence interval is (-0.94%, 1.81%)
Mean of AUC gain in cross-validation 5ch with EI index first vs 3ch is 0.39%. 95% Confidence interval is (0.07%, 0.71%)
Mean of AUC gain in validation on Kaggle Public test set 5ch with MI index first vs 3ch is 1.15%. 95% Confidence interval is (-0.12%, 2.42%)
Mean of AUC gain in validation on Kaggle Public test set 5ch with EI index first vs 3ch is 0.30%. 95% Confidence interval is (-0.04%, 0.64%)
Mean of AUC gain in validation on Kaggle Private test set n 5ch  with MI index first vs 3ch is 1.13%. 95% Confidence interval is (-0.68%, 2.93%)
Mean of AUC gain in validation on Kaggle Private test set n 5ch  with EI index first vs 3ch is 0.36%. 95% Confidence interval is (0.12%, 0.61%)

Mean of AUC gain in cross-validation 4ch with MI vs 3ch is -0.25%. 95% Confidence interval is (-0.59%, 0.09%)
Mean of AUC gain in cross-validation 4ch with EI vs 3ch is 0.63%. 95% Confide